In [23]:
from keras.models import Sequential
from keras.layers import Dense


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
# import librart
import sklearn
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
from scipy.sparse import csr_matrix
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import feature_select
import feture_process
from scipy.sparse import load_npz
from scipy.sparse import load_npz
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [24]:
# use countvectorizer
# use countvectorizer
# use countvectorizer
# use countvectorizer

# import data
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")

In [25]:
# process Prublisher and Language
train_df["Language"], test_df["Language"] = feture_process.docclass_preprocess(train_df["Language"],test_df["Language"],10)
train_df["Publisher"], test_df["Publisher"] = feture_process.docclass_preprocess(train_df["Publisher"],test_df["Publisher"],200)

In [26]:
# use oneHotCode for Publisher (sklearn)
publisher_train_hot = feture_process.process_OneHotEncoder(train_df,"Publisher")
publisher_test_hot = feture_process.process_OneHotEncoder(test_df,"Publisher")
train_df = pd.concat([train_df, publisher_train_hot], axis=1)
test_df = pd.concat([test_df, publisher_test_hot], axis=1)

In [27]:
# use oneHotCode for Language (sklearn)
language_train_hot = feture_process.process_OneHotEncoder(train_df,"Language")
language_test_hot = feture_process.process_OneHotEncoder(test_df,"Language")
train_df = pd.concat([train_df, language_train_hot], axis=1)
test_df = pd.concat([test_df, language_test_hot], axis=1)

In [28]:
# use oneHotCode for label (sklearn)
label_train_hot = feture_process.process_OneHotEncoder(train_df,"rating_label")
train_df = pd.concat([train_df, label_train_hot], axis=1)
train_df = train_df.drop(columns=["rating_label"])

In [47]:
# train_name_countvectorizer
train_name_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_name_countvectorizer.pkl", "rb"))
train_name_dic = train_name_countvectorizer.vocabulary_

# train_authors_countvectorizer
train_authors_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_authors_countvectorizer.pkl", "rb"))
train_authors_dic = train_authors_countvectorizer.vocabulary_

# train_desc_countvectorizer
train_desc_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_desc_countvectorizer.pkl", "rb"))
train_desc__dic = train_desc_countvectorizer.vocabulary_

# process vector features
train_name_features = train_name_countvectorizer.transform(train_df['Name'])
train_authors_features = train_authors_countvectorizer.transform(train_df['Authors'])
train_desc_features = train_desc_countvectorizer.transform(train_df['Description'])
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language','rating_label_3.0','rating_label_4.0','rating_label_5.0'])
train_label = train_df[['rating_label_3.0','rating_label_4.0','rating_label_5.0']]

In [48]:
# only use dense matrix
train_features = other_features_df

In [ ]:
# new sparse features
sparse_features = hstack([train_name_features, train_authors_features, train_desc_features])
# new train features
dense_features = csr_matrix(other_features_df.values)
train_features = hstack([sparse_features, dense_features])

In [49]:
# process test features
test_name_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_name_vec.npz')
test_authors_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_authors_vec.npz')
test_desc_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_desc_vec.npz')
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])



In [50]:
# only use dense matrix
test_features = test_other_features_df

In [ ]:
test_sparse_features = hstack([test_name_features, test_authors_features, test_desc_features])
new_dense_features = csr_matrix(test_other_features_df.values)
test_features = hstack([test_sparse_features, new_dense_features])

In [36]:
test_features.shape

(5766, 14)

In [52]:
input_shape = (train_features.shape[1], 1)  # 100个特征，每个特征1个通道
num_classes = train_label.shape[1]

model = Sequential()

# 添加第一个卷积层
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))

# 添加第二个卷积层
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# 将卷积结果扁平化，以便连接到全连接层
model.add(Flatten())

# 添加全连接层
model.add(Dense(64, activation='relu'))

# 添加输出层，使用softmax激活函数进行多分类
model.add(Dense(num_classes, activation='softmax'))

# 编译模型，使用分类交叉熵损失函数和Adam优化器
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 打印模型概览
model.summary()








Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 12, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 6, 32)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 64)             6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 2, 64)            0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8

In [53]:
X_train, X_val, y_train, y_val = train_test_split(train_features, train_label, test_size=0.3, random_state=66)

In [54]:
cnn = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
505/505 [==============================] - 1s 1ms/step - loss: 1.2362 - accuracy: 0.6176 - val_loss: 0.8677 - val_accuracy: 0.7063
Epoch 2/50
505/505 [==============================] - 1s 1ms/step - loss: 0.8588 - accuracy: 0.6502 - val_loss: 0.7402 - val_accuracy: 0.7070
Epoch 3/50
505/505 [==============================] - 1s 1ms/step - loss: 0.8188 - accuracy: 0.6557 - val_loss: 0.7584 - val_accuracy: 0.7073
Epoch 4/50
505/505 [==============================] - 1s 1ms/step - loss: 0.7835 - accuracy: 0.6703 - val_loss: 0.8118 - val_accuracy: 0.7078
Epoch 5/50
505/505 [==============================] - 1s 1ms/step - loss: 0.7701 - accuracy: 0.6796 - val_loss: 0.7108 - val_accuracy: 0.7067
Epoch 6/50
505/505 [==============================] - 1s 1ms/step - loss: 0.7376 - accuracy: 0.6927 - val_loss: 0.7603 - val_accuracy: 0.7079
Epoch 7/50
505/505 [==============================] - 1s 1ms/step - loss: 0.7454 - accuracy: 0.6933 - val_loss: 0.7471 - val_accuracy: 0.7072
Epoch 